In [ ]:
# Moving from many .json files with all processed photoelectrons information produced by jupyternotebook_esecuzione.ipynb
# to a single .json file for each energy
# with pe signal info channel by channel (both APA1 and APA2)

# The signal selection is done by looking at the APA 1 pe threshold (i choose it as the crossing point between the langauss and gaussian fit of the APA1 mean pe distribution)

# The new dictionary is saved as channel_data.json in the output folder (channel_study), energy per energy

In [ ]:
import importlib
import waffles.np04_analysis.lightyield_vs_energy.scripts.utils as utils_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils import *

In [ ]:
# Energy to analyze
energy = 7

energy_dict = {1: 0, 2: 118, 3: 113, 5: 174, 7: 238} # I'm using the intersection between the fitted langaauss and gaussian of the apa1 mean pe distribution
apa_mean_pe_threshold = energy_dict[energy]

input_folder = f"/afs/cern.ch/work/a/anbalbon/private/waffles/src/waffles/np04_analysis/lightyield_vs_energy/output/apa1_vs_apa2"

apa12_energy_folder = f"{input_folder}/{energy}GeV"
output_folder= f"{apa12_energy_folder}/channel_study"

In [ ]:
def load_energy_json_dict(energy: int, apa12_folder: str, output_dir: str) -> dict:
    """
    Legge tutti i JSON 'photoelectron_dic_{energy}GeV.json' nelle sottocartelle
    '{start}_to_{stop}' di {energy}GeV e li raccoglie in un unico dizionario:

        merged_dict["start_to_stop"] = contenuto_json

    Inoltre concatena tutti i files_read.txt in un unico file
    'files_read_ALL.txt' dentro output_dir.
    """

    energy_folder = Path(apa12_folder)

    if not energy_folder.exists():
        raise FileNotFoundError(f"Folder {energy_folder} doesn't exist")

    merged_dict = {}
    all_txt_lines = []

    for subfolder in sorted(energy_folder.iterdir()):
        # solo directory valide
        if not subfolder.is_dir():
            continue

        # solo cartelle tipo {start}_to_{stop}
        if "_to_" not in subfolder.name:
            continue

        json_file = subfolder / f"photoelectron_dic_{energy}GeV.json"
        txt_file = subfolder / "files_read.txt"

        # ---------- JSON ----------
        if not json_file.exists():
            print(f"⚠️ Missing JSON file: {json_file}")
            continue

        with json_file.open("r") as f:
            data = json.load(f)

        # salva il JSON sotto il nome della cartella
        merged_dict[subfolder.name] = data

        # ---------- TXT ----------
        if txt_file.exists():
            with txt_file.open("r") as f:
                # pulisce newline e mantiene ordine
                all_txt_lines.extend(line.rstrip() + "\n" for line in f)
        else:
            print(f"⚠️ Missing TXT file: {txt_file}")

    if not merged_dict:
        raise RuntimeError(f"No JSON files found for energy = {energy}")

    # ---------- SCRITTURA FILE TXT UNICO ----------
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # rimuove eventuali duplicati mantenendo l'ordine
    all_txt_lines = list(dict.fromkeys(all_txt_lines))

    output_txt = output_dir / "files_read_ALL.txt"
    with output_txt.open("w") as f:
        f.writelines(all_txt_lines)

    print(f"✅ Written merged txt file: {output_txt}")
    print(f"✅ Total txt lines: {len(all_txt_lines)}")
    print(f"✅ Total JSON blocks: {len(merged_dict)}\n")

    return merged_dict


In [ ]:
merged_dict = load_energy_json_dict(energy, apa12_energy_folder, output_folder)

In [ ]:
# SUMMARY OF THE DICTIONARY STRUCTURE

print("\nSUMMARY OF THE DICTIONARY STRUCTURE\n")

print(f"merged_dict keys:\n{list(merged_dict.keys())}\n")
print(f"merged_dict['0_to_10'] keys:\n{list(merged_dict['0_to_10'].keys())}\n")
print(f"merged_dict['0_to_10']['1'] keys:\n{list(merged_dict['0_to_10']['1'].keys())}\n")
print(f"merged_dict['0_to_10']['1']['channel_dic'] is a list of dict keys\n")
print(f"merged_dict['0_to_10']['1']['channel_dic'][0] keys: endpoints\n{list(merged_dict['0_to_10']['1']['channel_dic'][0].keys())}\n")
print(f"merged_dict['0_to_10']['1']['channel_dic'][0]['104'] keys: channels\n{list(merged_dict['0_to_10']['1']['channel_dic'][0]['104'].keys())}\n")
print(f"merged_dict['0_to_10']['1']['channel_dic'][0]['104']['0'] keys:\n{list(merged_dict['0_to_10']['1']['channel_dic'][0]['104']['0'].keys())}\n")

In [ ]:
# SELECTING TRIGGERS INDEXES WHERE APA1 MEAN PE > THRESHOLD

dic_trigger_index = {}
for _to_, to_dict in merged_dict.items(): # looking at each "0_to_10", "10_to_20", ...
    index_list = []
    for i, trigger_mean_pe in enumerate(to_dict['1']['mean']): # looking at each trigger
        try:
            if trigger_mean_pe > apa_mean_pe_threshold: # checking if the mean photoelectrons in APA1 is above threshold
                index_list.append(i)
        except:
            continue
    dic_trigger_index[_to_] = index_list # saving the list of trigger indexes that passed the threshold for this "_to_"


In [ ]:
channel_data = {104: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 1: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 3: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 4: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 6: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    10: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 11: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 12: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 13: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 14: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 15: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 16: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 17: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}},
            105: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 1: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 3: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 4: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 6: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    10: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 12: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 15: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 17: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 21: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 23: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 24: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 26: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}},
            107: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    10: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 12: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 15: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 17: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}},
            109: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 1: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 3: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 4: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 6: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    10: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 11: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 12: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 13: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 14: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 15: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 16: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 17: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    20: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 21: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 22: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 23: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 24: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 25: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 26: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 27: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    30: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 31: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 32: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 33: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 34: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 35: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 36: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 37: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    40: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 41: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 42: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 43: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 44: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 45: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 46: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 47: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}},
            111: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 1: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 3: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 4: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 6: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    10: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 11: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 12: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 13: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 14: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 15: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 16: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 17: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    20: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 21: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 22: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 23: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 24: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 25: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 26: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 27: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    30: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 31: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 32: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 33: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 34: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 35: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 36: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 37: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    40: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 41: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 42: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 43: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 44: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 45: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 46: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 47: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}},
            112: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 1: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 3: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 4: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 6: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    10: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 11: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 12: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 13: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 14: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 15: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 16: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 17: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    20: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 21: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 22: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 23: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 24: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 25: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 26: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 27: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    30: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 31: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 32: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 33: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 34: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 35: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 36: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 37: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []},
                    40: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 42: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 45: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 47: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}},
            113: {0: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 2: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 5: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}, 7: {'n_pe' : [], 'e_n_pe':  [], 'trigger time' : [], 'trigger index' : []}}}

In [ ]:
# EXTRAPOLATING SINGLE CHANNEL PHOTOELECTRON INFORMATION  

for _to_, index_list in dic_trigger_index.items(): # looking at each "0_to_10", "10_to_20", ...
    for index in index_list:
        # accessing channel_dic, "index" element containg all triggered channels for that beam trigger
        for apa in ['1', '2']: # looking at APA1 and APA2
            for end, end_dic in merged_dict[_to_][str(apa)]["channel_dic"][index].items(): # looking at each endpoint
                for ch, ch_data in end_dic.items(): # looking at each channel
                    channel_data[int(end)][int(ch)]['n_pe'].append(ch_data['n_pe'])
                    channel_data[int(end)][int(ch)]['e_n_pe'].append(ch_data['e_n_pe'])
                    channel_data[int(end)][int(ch)]['trigger index'].append(index)
                    channel_data[int(end)][int(ch)]['trigger time'].append(merged_dict[_to_]['trigger time'][index])

In [ ]:
with open(f"{output_folder}/channels_data.json", "w") as f:
    json.dump(channel_data, f, indent=4)

In [ ]:
# Print output
my_endpoint = 112
my_channel = 35
print(len(channel_data[int(my_endpoint)][int(my_channel)]['n_pe']))
print(channel_data[int(my_endpoint)][int(my_channel)]['n_pe'])